In [19]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

In [20]:
model = Network('model')

node = np.empty(4, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[2] = Source(model,'Source')
node[3] = Sink(model,'Sink')

jobclass = OpenClass(model, 'Class1', 0)

node[0].setService(jobclass, HyperExp(0.5,3.0,10.0))
node[1].setService(jobclass, Exp(1))
node[2].setArrival(jobclass, Exp(0.1))

M = model.getNumberOfStations()
K = model.getNumberOfClasses()

P = model.initRoutingMatrix()
P.set(jobclass, jobclass, [[0,1,0,0],[0,0,0,1],[1,0,0,0],[0,0,0,0]])

model.link(P)

This example shows the execution of the solver on a 1-class 2-node open model.

In [21]:
#options = Solver.defaultOptions
#options.keep = True
#options.verbose = 1
#options.cutoff = 10
#options.seed = 23000
#options.iter_max = 200
#options.samples = 2000

# This part illustrates the execution of different solvers
solver = np.array([], dtype=object)
solver = np.append(solver, SolverCTMC(model,'keep',True,'cutoff',1))
solver = np.append(solver, SolverJMT(model,'seed',23000,'verbose',False,'keep',True))
solver = np.append(solver, SolverSSA(model,'seed',23000,'verbose',True,'samples',10000))
solver = np.append(solver, SolverFluid(model))
solver = np.append(solver, SolverMVA(model))
solver = np.append(solver, SolverNC(model))
#solver = np.append(solver, SolverMAM(model)) # TODO: bug in MAM
#solver = np.append(solver, LINE(model))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].getName()}')
    AvgTable[s] = solver[s].getAvgTable()


SOLVER: SolverCTMC
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.0193  0.0193  0.2167  0.2167  0.0892  0.0892
1  Queue1   Class1  0.0892  0.0892  1.0000  1.0000  0.0892  0.0892
2  Source   Class1  0.0000  0.0000  0.0000  0.0000  0.0000  0.0892

SOLVER: SolverJMT
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.0227  0.0227  0.2148  0.2148  0.0996  0.0996
1  Queue1   Class1  0.1133  0.1022  1.1075  1.1075  0.0996  0.1000
2  Source   Class1  0.0000  0.0000  0.0000  0.0000  0.0000  0.0996

SOLVER: SolverSSA
SSA samples:   10000
SSA [method: default(serial), lang: java, env: 21.0.7] completed in 0.673000s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.0215  0.0215  0.2196  0.2196  0.1000  0.0981
1  Queue1   Class1  0.1086  0.0981  1.1105  1.1105  0.0981  0.0978
2  Source   Class1  0.0000  0.0000  0.0000  0.0000  0.0000  0.1000

SOLVER: SolverFluid
Fluid [method: cl

java.lang.RuntimeException: The load-dependent solver does not support open classes yet.
	at jline.solvers.nc.handlers.Solver_ncldKt.solver_ncld(Solver_ncld.kt:56)
	at jline.solvers.nc.analyzers.Solver_ncld_analyzerKt.solver_ncld_analyzer(Solver_ncld_analyzer.kt:62)
	at jline.solvers.nc.SolverNC.runAnalyzer(SolverNC.java:249)
	at jline.solvers.NetworkSolver.getAvg(NetworkSolver.java:266)
	at jline.solvers.NetworkSolver.getAvgTable(NetworkSolver.java:592)


java.lang.RuntimeException: java.lang.RuntimeException: Unable to compute results and therefore unable to print AvgTable.